Fine-tuning best BART 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of the BART model on the new extracted sentences from the book **Grammaire de Wolof Moderne**. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import BartModel, BartForConditionalGeneration, Seq2SeqTrainer, BartTokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup,\
                          get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## Wolof to French

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = BartTokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/bart_tokenizers/tokenizer_v3.json")

In [3]:
def recuperate_datasets(wf_char_p: float, wf_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)

  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        corpus_1 = 'wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2,
                                        add_bos_token=True,
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        add_bos_token=True,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the evaluation function ⚙️

Let us initialize the evaluation object.

In [4]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [5]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
# from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5 2000

In [13]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.20442820063300543,
    'fr_word_p': 0.33807729221615707,
    'learning_rate': 0.0017263972464687655,
    'weight_decay': 0.015176927058663345,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 656,
    'epochs': 66,
    'mid_epoch': 66,
    'max_len': 21,
    'end_mark': 3,
    'bleu': 6.6232,
    'model_dir': 'data/checkpoints/wf_bart_custom_train_v3_checkpoints',
    'new_model_dir': 'data/checkpoints/bart_custom_train_results_wf_v3'
}

# Initialize the model name
model_name = 'facebook/bart-base'

# import the model with its pre-trained weights
model = BartForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 3, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/bart_custom_train_wf"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### --- Linear

In [12]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/61 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0017101135834161351]}


Test batch number 11: 100%|██████████| 11/11 [00:04<00:00,  2.30batches/s]



Metrics: {'train_loss': 0.8962184161315729, 'test_loss': 0.9789676666259766, 'bleu': 7.7285, 'gen_len': 9.4444}




  2%|▏         | 1/61 [00:37<37:26, 37.45s/it]

...

 98%|█████████▊| 60/61 [40:55<00:40, 40.19s/it]

For epoch 66: 
{Learning rate: [0.0015502376188994012]}


Test batch number 11: 100%|██████████| 11/11 [00:06<00:00,  1.61batches/s]



Metrics: {'train_loss': 0.024802282550314695, 'test_loss': 1.5957979614084417, 'bleu': 23.761, 'gen_len': 10.7716}




100%|██████████| 61/61 [41:35<00:00, 40.90s/it]


### --- Cosine

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### Predictions and Evaluation

In [14]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True, add_bos_token=True)

Let us make the evaluation and print the predicted sentences.

In [15]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 11: 100%|██████████| 11/11 [00:28<00:00,  2.56s/batches]

predictions_: [[2, 0, 319, 12, 40, 38, 716, 6, 2, 1, 1, 1, 1, 1, 1, 1], [2, 0, 319, 19, 1078, 223, 59, 109, 15, 470, 67, 1764, 26, 283, 16, 2], [2, 0, 1920, 15, 518, 936, 172, 748, 6, 2, 1, 1, 1, 1, 1, 1], [2, 0, 1678, 20, 171, 5, 63, 47, 5, 63, 322, 506, 6, 2, 1, 1], [2, 0, 1686, 59, 32, 317, 128, 9, 60, 147, 16, 2, 1, 1, 1, 1], [2, 0, 319, 19, 118, 12, 18, 40, 38, 6, 2, 1, 1, 1, 1, 1], [2, 0, 1738, 218, 48, 817, 223, 6, 2, 1, 1, 1, 1, 1, 1, 1], [2, 0, 1852, 636, 9, 25, 47, 67, 495, 26, 66, 70, 16, 2, 1, 1], [2, 0, 1920, 15, 1919, 491, 10, 24, 29, 11, 591, 19, 5, 8, 2, 1], [2, 0, 1920, 15, 1619, 1497, 1805, 252, 9, 278, 9, 5, 8, 2, 1, 1], [2, 0, 1885, 1203, 10, 47, 190, 964, 20, 6, 2, 1, 1, 1, 1, 1], [2, 0, 319, 12, 40, 38, 24, 368, 347, 16, 2, 1, 1, 1, 1, 1], [2, 0, 1568, 59, 591, 72, 184, 92, 130, 6, 2, 1, 1, 1, 1, 1], [2, 0, 1568, 53, 26, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 1686, 5, 9, 847, 317, 6, 2, 1, 1, 1, 1, 1, 1, 1], [2, 0, 1568, 1800, 690, 805, 62, 29, 102, 6, 2, 1, 1, 

In [16]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
151,Ñii dañu demul xanaa!,Ceux-ci ne sont peut-être pas partis!,Ceux-ci ne partent peut-être pas!
152,Keneen a ŋgi.,En voilà un autre.,En voilà un.
153,"Bon nag, maa ŋgi dem.","Dans ce cas, je pars.","Alors, ne partez pas."
154,Ana ŋgoor?,Où est NGor?,Où est celui-là?
155,"Defe naa du kookuu, du kookee.","Je crois que ce n'est pas celui-ci, ce n'est p...",Je reconnais les vaches d'Europe.
156,Suñu xarit la!,C'est leur ami!,C'est notre ami!
157,Xale bi ne ŋgeen dugg mu gis leen.,L'enfant dit d'entrer pour qu'il vous voie.,L'enfant dit d'entrer.
158,Gis naa góor gi doon wax te toogoon ci suuf.,J'ai vu cet homme qui parlait et qui était ass...,J'ai vu cet homme qui parlait.
159,Dimëlé leen ŋgir ñooñe!,Aide-les à cause de ceux-là!,Allez là-bas!
160,Mi ŋgiy laaj ndax demkoon ŋga.,Il demande si tu aurais été.,"Il est là, peut-être."


In [17]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
76,Waxtaan ak kenn kan?,Converser avec lequel?,Qui cherches-tu?
47,Góor gii demoon.,Cet homme qui était parti.,Cet homme qui a été.
94,Nileen ka.,Dites-lui.,Dites-leur.
158,Gis naa góor gi doon wax te toogoon ci suuf.,J'ai vu cet homme qui parlait et qui était ass...,J'ai vu cet homme qui parlait.
38,May naa ka keneen ku ka gën.,Je l'ai donné à quelqu'un de meilleur que lui.,J'ai donné le mouton mâle.
91,Ci keneen ki ci buntu kër gi.,Vers cet autre qui est devant la porte de la m...,Chez l'autre qui est dehors.
15,Góor gi nee na soo demee mi ŋgi fi.,L'homme dit qu'il serait peut-être là.,L'homme affirme qu'il est là.
113,Gis ŋga jigéen ñeneen ñooñu?,As-tu vu ces autres femmes en question?,Tu vois ces femmes-là?
64,Kookule la ci biir.,C'est celui-là à l'intérieur.,C'est celui-là qui est dehors.
57,Dil nitu réew mi!,Sois homme de ce pays!,Sois un homme de son!
